In [1]:
# IMPORT Thư viện
#  pip install numpy pandas jupyter notebook openpyxl xlrd
import pandas as pd
import os
import openpyxl
import xlrd
import numpy as np

In [2]:
# IMPORT CÁC FILE (5 FILES): TỔNG NHẬP XUẤT TỒN ; GIÁ BÁN; MUA HÀNG ; BÁN HÀNG; INPUT

# IMPORT 1 Tổng hợp nhập xuất tồn -> RENAME 
Data_Ton = pd.read_excel("Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A,F,K:L",skipfooter=1,header=5)
Data_Ton=Data_Ton.rename(columns={"Unnamed: 0":"ID",
                                "Unnamed: 5":"ID_Category",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất" })

#Import INPUT Quotation
Data_Request=pd.read_excel("Input.xlsx",usecols="B:H")

#Import 2: Báo cáo giá bán tồn kho
Data_GiaBanTonKho=pd.read_excel("Báo cáo giá bán, tồn kho.xls",usecols="C,F,J",header=4,skipfooter=1)
Data_GiaBanTonKho=Data_GiaBanTonKho.rename(columns={"Mã số":"ID"})
Data_GiaBanTonKho.dropna(how='all',inplace=True)


#Import 3: Sổ chi tiết bán hàng
Data_BanHang=pd.read_excel("Sổ chi tiết bán hàng.xls",usecols="F,M",skipfooter=1,header=6)
Data_BanHang=Data_BanHang.rename(columns={"Unnamed: 0":"Ngày bán",
                                        "Unnamed: 5":"ID" ,
                                        "Giá bán":"Giá bán giảm"})


#Import 4: Sổ chi tiết mua hàng
Data_MuaHang=pd.read_excel("Sổ chi tiết mua hàng (2).xls",usecols="A,C,F,I:J",header=5,skipfooter=1)
Data_MuaHang=Data_MuaHang.rename(columns={"Unnamed: 0":"Ngày nhập",
                                        "Unnamed: 2":"Nhà cung cấp",
                                        "Unnamed: 5":"ID",
                                        "Số lượng":"Tổng nhập",
                                        "Đơn giá":"Đơn giá nhập"})


In [3]:
# TẠO BỘ LỌC CHO SỔ MUA HÀNG: GIÁ NHẬP NHỎ NHẤT; NGÀY NHẬP GẦN NHẤT
# FILTER: Đơn giá nhập >0; Tổng nhập > 0; 
Data_MuaHang=Data_MuaHang.loc[(Data_MuaHang['Đơn giá nhập']>0) & (Data_MuaHang['Tổng nhập']>0)].sort_values('Đơn giá nhập')
# Tạo giá nhập nhỏ nhất
Min_value=Data_MuaHang.groupby('ID')['Đơn giá nhập'].min()
Data_MuaHang=Data_MuaHang.merge(Min_value,on='ID',suffixes=('', '_min'))
# -> Filter theo ID có đơn giá nhập nhỏ nhất
Data_MuaHang = Data_MuaHang[Data_MuaHang['Đơn giá nhập']==Data_MuaHang['Đơn giá nhập_min']].drop('Đơn giá nhập_min', axis=1)

#-> Filter Mã hàng có ngày nhập gần nhất
Date_max=Data_MuaHang.groupby('ID')['Ngày nhập'].max()
Data_MuaHang=Data_MuaHang.merge(Date_max,on='ID',suffixes=('', '_max'))
# -> Filter theo ID có đơn giá nhập nhỏ nhất
Data_MuaHang = Data_MuaHang[Data_MuaHang['Ngày nhập']==Data_MuaHang['Ngày nhập_max']].drop('Ngày nhập_max', axis=1)


# Data_MuaHang.loc[Data_MuaHang['ID']=="90913-02105"]
# Data_MuaHang.loc[Data_MuaHang['ID']=="T24NC50WB"]

In [4]:
# TẠO BỘ LỌC CHO SỔ BÁN HÀNG: GIÁ GIẢM NHỎ NHẤT;
Data_BanHang = Data_BanHang[Data_BanHang['Giá bán giảm'] == Data_BanHang.groupby('ID')['Giá bán giảm'].transform('min')]
Data_BanHang = Data_BanHang.drop_duplicates(['ID','Giá bán giảm'], keep="first")


In [5]:
# TÍNH TOÁN GIÁ VỐN
# Calcuation on Tong Hop Ton Kho -------------------------------------------

# Gia Von= Gia Tri Xuat/SL Xuat * 1.05
Data_Ton['Giá vốn']=np.around(Data_Ton['Giá trị xuất']/Data_Ton['Tổng xuất']*1.05,0)


In [6]:
# TỔNG HỢP DỮ LIỆU

# Merge Request -> Gia Von
Data_Request= pd.merge(Data_Request,Data_Ton[['ID','Giá vốn']],on='ID', how='left')
# # Merge  (Báo cáo nhập xuất tồn)
Data_Request=pd.merge(Data_Request,Data_GiaBanTonKho[['ID','Giá bán','Tổng tồn']],how='left',on='ID')
# # # # Merge Giá bán hàng
Data_Request=pd.merge(Data_Request,Data_BanHang,how='inner',on='ID')
# # Merge Giá mua hàng
Data_Request=pd.merge(Data_Request,Data_MuaHang,how='left',on='ID')


In [7]:
# TÍNH TOÁN CÁC HỆ SỐ
# Calcuation on Request -------------------------------------------

#VND/US=24,000; he so = 1.13
Data_Request['Unit Price VND'] = np.around(Data_Request['Unit price']*24000*1.13,0)
Data_Request['Tỷ lệ Giá vốn chênh lệch'] = np.around((Data_Request['Unit Price VND']-Data_Request['Giá vốn'])/Data_Request['Giá vốn'],2)

In [8]:
# MẶC CẢ GIÁ VỚI NHÀ CUNG CẤP; RATE = 15%
RateDeal=0.15
RateVNDUSD=24000

# GIÁ MẶC CẢ VND
Data_Request['Giá mặc cả VND']=np.around(Data_Request.loc[Data_Request['Tỷ lệ Giá vốn chênh lệch']<0.0000]['Giá vốn'],0)
Data_Request['Giá mặc cả VND']=np.around(Data_Request[Data_Request['Giá vốn']>=0.0]['Giá vốn']*(1-RateDeal),0)
Data_Request.sort_values(by='Giá mặc cả VND', ascending=False)

#GIÁ MẶC CẢ USD
Data_Request['Giá mặc cả USD']=np.around(Data_Request['Giá mặc cả VND']/RateVNDUSD,2)

# Data_Request.head()


In [9]:
#SHOW KET QUA
Data_Request=Data_Request.iloc[:,[0,1,2,3,4,5,6,15,7,16,8,10,11,12,14,13,9,17,18]]
#Unit Price, Unit VND, Giá vốn, %chênh lệch VND, Giá bán, giá giảm, Ngày nhập, Nhà cung cấp, đơn giá nhập, tổng nhập, tổng tồn
# Data_Request.head()

# Data_Request.loc[Data_Request['ID']=="T24VG4WS"]


In [10]:
# EXPORT FILE DATA
with pd.ExcelWriter('Report.xlsx') as writer:
    Data_Request.to_excel(writer, sheet_name='Data')
  



In [11]:
Data_Request

,ID,ID Item,Description,PCS/PACK,Min qty,Qty request,Unit price,Unit Price VND,Giá vốn,Tỷ lệ Giá vốn chênh lệch,Giá bán,Giá bán giảm,Ngày nhập,Nhà cung cấp,Đơn giá nhập,Tổng nhập,Tổng tồn,Giá mặc cả VND,Giá mặc cả USD
0,T24NC50WB,48655-0D140,"Low. arm bushing VIOS NCP150,YARIS NSP152,SIEN...",4,72,3000,6.03,163534.0,135267.0,0.21,188000.0,165000.0,2022-05-13,NK-CHI,124956.0,100.0,0.0,114977.0,4.79
1,T24NC93WB,48655-0D051,"Low. arm bushing VIOS NCP93,YARIS NCP91",4,72,300,4.34,117701.0,100138.0,0.18,121000.0,110000.0,NaT,NaN,NaN,NaN,6.0,85117.0,3.55
2,T24S01WS,48654-16110,Low. arm bushing SOLUNA SMALL,10,180,100,2.61,70783.0,61716.0,0.15,81000.0,81000.0,NaT,NaN,NaN,NaN,16.0,52459.0,2.19
3,T24NC42WB,48655-0D060,"Low. arm bushing VIOS,ECHO,YARIS,VITZ BIG",4,72,100,3.79,102785.0,90730.0,0.13,110000.0,110000.0,NaT,NaN,NaN,NaN,4.0,77120.0,3.21
4,T24VG4WS,48654-0K040,"Low. arm bushing HILUX VIGO 4WD,FORTUNER SMALL",4,72,500,6.40,173568.0,144213.0,0.20,210000.0,160000.0,2022-05-13,NK-CHI,132678.0,100.0,83.0,122581.0,5.11
5,T24ZE121B,48655-12170,"Low. arm bushing ALTIS ZZE121,122,141 BIG",4,72,1000,4.34,117701.0,103401.0,0.14,158000.0,110000.0,NaT,NaN,NaN,NaN,0.0,87891.0,3.66
6,T24C02WB,48655-33040,Low. arm bushing CAMRY SXV10 (2.2 CC) (1994) BIG,6,108,54,3.15,85428.0,98771.0,-0.14,150000.0,140000.0,2022-05-13,NK-CHI,94068.0,12.0,0.0,83955.0,3.50
7,D24BT50W,UC3C 34 460A,"Low. arm bushing BT50 PRO 2WD,4WD",4,72,500,7.04,190925.0,164299.0,0.16,210000.0,200000.0,NaT,NaN,NaN,NaN,0.0,139654.0,5.82
8,D2441XW,UH74 34 450,"Low. arm bushing FIGHTER,RANGER (4WD) SMALL",8,144,200,2.33,63190.0,58040.0,0.09,80000.0,75000.0,NaT,NaN,NaN,NaN,9.0,49334.0,2.06
9,T25C04B,48725-48020B,"Suspension bush CAMRY (2002) ACV30,(2007) ACV4...",6,108,504,2.93,79462.0,77912.0,0.02,97000.0,90000.0,2022-05-13,NK-CHI,62478.0,12.0,0.0,66225.0,2.76
